## PyO3

In [1]:
%pip uninstall felsenstein-rs -y
!maturin build --release
%pip install target/wheels/felsenstein_rs-0.1.0-cp311-cp311-manylinux_2_34_x86_64.whl

Found existing installation: felsenstein-rs 0.1.0
Uninstalling felsenstein-rs-0.1.0:
  Successfully uninstalled felsenstein-rs-0.1.0
Note: you may need to restart the kernel to use updated packages.
🔗 Found pyo3 bindings
🐍 Found CPython 3.11 at /home/conda/miniconda3/envs/jax/bin/python3
📡 Using build options features from pyproject.toml
   --> src/lib.rs:261:13
    |
261 |         let grad_log_prior_total: Vec<[Float; Entry::DIM]>;
    |             ^^^^^^^^^^^^^^^^^^^^ help: if this is intentional, prefix it with an underscore: `_grad_log_prior_total`
    |
    = note: `#[warn(unused_variables)]` on by default

    Finished release [optimized] target(s) in 0.03s
📦 Built wheel for CPython 3.11 to /home/jora/src/rust/felsenstein-rs/target/wheels/felsenstein_rs-0.1.0-cp311-cp311-manylinux_2_34_x86_64.whl
Defaulting to user installation because normal site-packages is not writeable
Processing ./target/wheels/felsenstein_rs-0.1.0-cp311-cp311-manylinux_2_34_x86_64.whl
Note: you may need to

In [2]:
import numpy as np
from itertools import combinations
import felsenstein_rs as frs

In [3]:
# This now gives the correct error message
#ftree_bad = frs.FTree("data/tree_topological.csv", 1e-4)

In [4]:
ftree = frs.FTree("data/tree.txt", 1e-4)

In [5]:
all_index_pairs = np.array([[*tup] for tup in combinations(range(119), 2)], dtype=np.uint64)

In [6]:
# TODO accept dtype=int instead of uint
index_pairs = np.array([[0, 1], [0, 2], [0, 3]], dtype=np.uint64)
#index_pairs = all_index_pairs

In [7]:
symmetric_matrix = -np.eye(16) + (np.ones((16, 16)) - np.eye(16)) * (1/15)
symmetric_matrices = np.broadcast_to(symmetric_matrix, (index_pairs.shape[0],) + symmetric_matrix.shape)

In [8]:
sqrt_pi = np.random.random(16)**2

In [9]:
sqrt_pi_total = np.broadcast_to(sqrt_pi, (index_pairs.shape[0],) + np.ones(16).shape)

In [10]:
rate_matrix = np.diag(1/sqrt_pi) @ symmetric_matrix @ np.diag(sqrt_pi)
rate_matrices = np.broadcast_to(rate_matrix, (index_pairs.shape[0],) + rate_matrix.shape)

In [12]:
result_old = ftree.infer(index_pairs, rate_matrices, 2 * np.log(sqrt_pi_total))

In [13]:
result_new = ftree.infer_param(index_pairs, symmetric_matrices, sqrt_pi_total)

In [14]:
result_old[1][0] - result_new[3][0]

array([[-8.31279490e-15, -5.13478149e-16, -9.67975700e-16,
         1.14530607e-12, -5.80091530e-15, -1.86656246e-15,
        -6.21724894e-15, -2.15105711e-16, -1.95399252e-14,
        -1.88737914e-13, -7.44543316e-15, -1.70419234e-14,
        -1.51961776e-15,  6.24500451e-16, -1.70974346e-14,
        -5.71764858e-15],
       [-6.14353013e-12, -6.97220059e-13, -6.10400619e-13,
        -1.12686394e-09, -7.78399567e-12,  7.06101844e-13,
        -9.18376486e-12, -6.13065154e-12, -6.37783160e-11,
        -5.43195711e-10, -2.75601764e-12, -1.41398004e-10,
        -6.00852701e-13, -5.79536419e-13, -1.73301373e-11,
        -1.58451030e-12],
       [ 1.38697942e-11,  9.49285095e-12,  1.06368248e-11,
        -1.16073352e-07,  1.99520400e-11, -6.39488462e-14,
         8.91304808e-11, -7.50333129e-12,  8.58562998e-10,
         3.23925633e-08,  2.62616595e-11,  2.13731255e-11,
        -2.55795385e-13, -6.52988774e-12, -7.73070497e-11,
         8.20818968e-11],
       [-1.80841453e-13, -2.02676417e